## マウント，インストール

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade gspread
!pip install openai
!pip install emoji
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.4 MB/s eta 0:00:00


## レビュー準備

In [ ]:
import pandas as pd
import emoji

# CSVファイルの読み込み
data = pd.read_csv('drive/MyDrive/review/review_ver20250108.csv')
review_label_list = data.values.tolist()


#dataset作成
review_list = []
label_list = [] #onehot
one_cel_label_list = []
usefull_num = []
label_num = 8 #分類カテゴリ数
each_label_num = {} #各label数のカウント
#each_label_numのvalueを初期化
for i in range(label_num):
  each_label_num[i+1] = 0

for i in range(len(review_label_list)):
  review = emoji.replace_emoji(review_label_list[i][1])
  review_list.append(review)
  usefull_num.append(review_label_list[i][4])
  one_cel_label_list.append(review_label_list[i][2])
  str_category_list = review_label_list[i][2].split(',')
  try:
    category_list = [int(num) for num in str_category_list]
  except ValueError:
    print(i)
  onehot_label = []
  #空のマルチラベルリスト作成
  for i in range(label_num):
    onehot_label.append(0)
  for category in category_list:
    onehot_label[int(category)-1] = 1
    each_label_num[int(category)] =  each_label_num[int(category)] + 1
  label_list.append(onehot_label)
print(each_label_num)

{1: 659, 2: 99, 3: 171, 4: 264, 5: 25, 6: 26, 7: 22, 8: 6}


In [ ]:
def show_category_to_review(label_list, review_list, category_num):
  """
  各カテゴリーに属するレビューをprintする関数
  label_list : ワンホット表現のラベルリスト [[0,0...0],[0,0,..1]...]
  review_list : レビューリスト　['aaa', 'bbb'...]
  category_num : 探したいカテゴリーナンバー　1〜9
  """
  category_review_dic = {}
  for i in range(len(label_list)):
    if label_list[i][category_num-1] == 1:
      #print(f'ID : {i+1} \n review : {review_list[i]}\n label : {label_list[i]} \n usefull_num : {usefull_num[i]}\n')
      category_review_dic[i+1] = review_list[i]

  return category_review_dic

category_review_dic = show_category_to_review(label_list, review_list, 2)

In [ ]:
print(category_review_dic)

{3: 'ないと困る\n5~6年前買った羽根型使ってたのが壊れ購入\n断然こちらの方が、きっちり顔に風が来ていいです\n内側の丸い部分は風が出ないとレビュー書いて下さってた意味使って理解てきた\nそこが少し残念なのと\n説明書に保証延長のLINE登録書いてあったけれど、問い合わせも登録も全く存在してない感じなので保証期間内に壊れたら心配\n ', 11: '\nリピートしたけど…違うブランドになっちゃって…比べてみたら…前の方が風が強くて…\n俺はまあまあかな', 14: 'とても良い商品です。\nモノが意外と大きく仕事中の使用はできないので、プライベート用か休憩時に使う事にしてます\n\n電池は案外保つけど、最大風力だと微妙です\n\nそれでも首から上を冷すにはかなり涼しいので、重宝しそうです\n\n大きくて少し重いし、首に掛けてる時に下手に動くと外れそうなので、星4つとします', 30: '他機種も購入したけど\n同時に、同様な品物、機種は異なりますが、こちらの方が風量は弱でも強い気がしました。\n充電の持ちも一度のフル充電で往復３時間強通勤時間\n充分持ちます。\n', 32: 'まぁまぁかな？\nただ先端にファンがついてるだけのものとは違いますが\n値段相応といったところでしょうか。\n', 54: '金額以上にいいかも\n他商品と比べて安かったけど、レビューもいいので？セールで購入しました。\nちょっと大きいかなと思ったけど、\n見た目より重さは気にならなかったです。\n最大にしても、音はそんなに気にならない。\n\nまだ外で利用していませんが楽しみです\n', 56: '充電早い、バッテリー長い\nちょっと前にAmazon購入の同じタイプと比べて、バッテリー長持ちで、充電終了も早く、暑がりの私には助かっています。冷却部分は冷たい感触はありません、ちょっと前のは、良くずれて風が当たらないことありですが、とてもフィット感あり、ただし猛暑の折外では強目でないと、風感じません。\n予備で購入でしたが、前のはお蔵入りになりそうです。\n', 71: 'ペルチェ素子かと思ったら違った\nただの首掛け扇風機だった', 77: '失敗\n「ネックファン\u3000冷却プレート」で検索している内に海の物とも山の物ともわからないので最初は安いの買ってみるかと思い出したら本品

## ChatGPT api

#### 重要表現

In [ ]:
# 比較表現
comparative_expression1 = """
〜と比べると〜
〜より良い，悪い，高い，低い
〜の方がよかった。
前のは〜だけど，これは〜
今まで〜だったけど〜してよかった
"""
comparative_expression2 = """
〜で1番〜だった。
〜なこちらを選んだ。
〜が前のものより〜になった。
〜が以前より〜だった。
〜の様に〜ならよかった
"""
comparative_expression3 = """
〜と悩みましたが、こちらの商品にしました
〜という嬉しい改良点がありました。
前のモデルは〜で本商品に買い替えました。
他社の〜の場合、〜だった。この商品は〜だ。
〜の商品より〜倍〜です。
"""
comparative_expression4 = """
〜断然こちらの方が〜
〜比べてみたら〜
ただ〜のものとは違いますが〜
他商品と比べて〜
〜タイプと比べて〜
"""

#簡易表現
simple_expression1 = """

"""

### API実装

In [ ]:
from sre_compile import INFO
from openai import OpenAI
def generate_aug_review(generate_num, review, aug_method_num, aug_label, important_expression=comparative_expression1):

  if aug_label == 2:
    label_definition = """
    他の商品や同じ商品の前のバージョンと比較を行なっている文が存在する場合，このラベルが該当する．
    """
    label_name = "比較"

  elif aug_label == 5:
    label_definition = """
    良い，悪いなどの簡易的な表現や嬉しい，楽しいなどの単純な表現や感情表現のみで構成されており，具体的な評価対象が含まれていない場合，このラベルが該当する．
    この簡易表現ラベルでは，評価の根拠となる商品の機能面（使いやすさ，使用感，サイズ，デザインも含む）や価格，配送，商品を利用する状況（使用する頻度や活用方法）の全てについても触れていないことが条件となる．
    「音質が良い」や「履き心地が良い」，「デザインが良い」などは良い理由となる根拠が含まれているため簡易表現ラベルに該当しない．
    """
    label_name = "簡易表現"

  elif aug_label == 6:
    label_definition = """"
    商品の状態が良い，悪い，傷ついているなどの表現を含む文が存在する場合，このラベル
    が該当する.また，商品に対するサービス対応のレビューもこのラベルに該当する.
    """
    label_name = "商品状態"

  elif aug_label == 7:
    label_definition = """
    商品の配送が早い，遅いなどの表現を含む文が存在する場合，このラベルが該当する．
    """
    label_name = "商品配送"

  message1 = [
        {"role": "user", "content": [{"type": "text", "text":
                                      f"""以下はECサイトのレビュー1文です。このレビューを言い換える、または文節を入れ替えて文全体として同様の意味を保持したレビューを{generate_num}文生成してください。生成された{generate_num}文のレビューが文全体として同様の意味を持つことを厳守してください。
                                      {review}
                                      """}]},
  ]
  message2 = [
        {"role": "user", "content": [{"type": "text", "text":
                                      f"""ECサイトのレビューを利用してファインチューニングを行っています。{label_name}表現を用いたレビューのデータ拡張を行いたいです。以下の{label_name}を利用してレビューを{generate_num}文作成してください。
                                      {important_expression}
                                      """}]},
  ]
  message3 = [
        {"role": "user", "content": [{"type": "text", "text":
                                      f"""
                                      ECサイトレビューにおける1つのラベルの定義が下記です。
                                      {label_definition}
                                      以下は探索材（電化製品，衣類等）における{label_name}ラベルを含むレビュー例です。このように{label_name}を含む文章を{generate_num}文生成してください。また条件として以下のものを追加します。
                                      １．レビュー例の商品に限らず，別の商品（探索材）のレビューでも構わないです。 ２．ラベルの特徴を反映させたレビューであることは厳守してください。\n
                                      {review}
                                      """}]},
  ]

  if aug_method_num == 1:
    message = message1
  elif aug_method_num == 2:
    message = message2
  elif aug_method_num == 3:
    message = message3

  #api key
  client = OpenAI(api_key="APIkeyを入力")
  response = client.chat.completions.create(
      model="gpt-4o-mini-2024-07-18",
      messages=message,
      temperature=0.71,
      max_tokens=16383,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
  )
  aug_review = response.choices[0].message.content
  return aug_review, message


<ipython-input-8-19d783fd1af8>:1: DeprecationWarning: module 'sre_compile' is deprecated
  from sre_compile import INFO


### api実装確認

In [ ]:
#手法1の試し
r1 =""""
とても良い商品です。
・風力は電源ボタンを押す度に変更可能です。
・本日日中に使用しましたところ、手持ちの扇風機よりかなり風量があり、両手が開くので楽です。
・羽なしとありますが、本体内部に直径4～5㎝程の風車が左右に各1個内蔵されていますので、見た目は
　羽なしです。
・首の後ろに当たる部分に金属プレートが付いていますが、初めはひんやりしますが、そのうち体温か
　外気温で温かくなってしまいました。私の使い方が悪かったのかな。
・バッテリーについては、申し分ないと思います。ただし室外の長時間作業で使用するとなると考えて
　使用しないといけないかと思います。
・通勤通学で1時間、往復2時間と仮定し、月～金まで5日間使用して約10時間位、土日に充電する。
　という使用サイクルになるかと思います。
・あとは全体的に耐久性がどのくらいあるか期待しています。
"""
#aug_r1, m = generate_aug_review(5, r1, 1, 2)
#print(aug_r1)

In [ ]:
# 手法2の試し
#aug_r2, m = generate_aug_review(10, r1, 2, 2, comparative_expression1)
#print(aug_r2)

In [ ]:
#  手法3の試し
#aug_r3, m = generate_aug_review(10, r1, 3, 2)
#print(aug_r3)

## テキスト処理

In [ ]:
from inspect import cleandoc
import re

def devide_text(text, generate_num, id):
  """
  # レビューを1つずつ分割する（改行で区切る）
  reviews = text.strip().split("\n")
  # 冒頭の数字とピリオドと空白を削除し、結果をリストにする
  cleaned_reviews = [re.sub(r'^\d+\.\s*', '', review) for review in reviews]
  # 空文字列を削除
  cleaned_reviews = [review for review in cleaned_reviews if review.strip() != ""]
  """
  # 数字とピリオドで始まるレビュー全体を抽出
  reviews = re.findall(r"\d+\.\s(.*?)(?=(?:\n\d+\.|\Z))", text, re.DOTALL)
  # レビューが見つからない場合は、そのままテキストをリストとして返す
  if not reviews:
    return [text.strip()]

  # 各レビューから冒頭の余計な改行や空白を除去
  cleaned_reviews = [review.strip() for review in reviews if review.strip()]


  if len(cleaned_reviews) == generate_num:
    return cleaned_reviews
  elif len(cleaned_reviews) != generate_num:
    print(f'ID : {id}\n {generate_num}分割失敗')
    return cleaned_reviews

#print(devide_text(text, 5))

In [ ]:
# devide_text のテスト
#print(devide_text(aug_r1, 5, 3))

In [ ]:
# devide_text のテスト
#print(devide_text(aug_r2, 10, 3))

## スプレッドシート記述定義

In [ ]:
import gspread
from google.auth import default
import re
import time
from tqdm import tqdm
import math

# Google Sheets APIのクライアントを設定
creds, _ = default()
gc = gspread.authorize(creds)
title_text = "5つ星のうち2.5"
def write_reviews_to_sheet(id, sheet_name, reviews, one_cel_label_list):
    # スプレッドシートへのアクセス
    workbook = gc.open_by_key(id)
    worksheet = workbook.worksheet(sheet_name)

    # 既存データの取得
    data = worksheet.get_all_values()[1:]  # ヘッダーを除外

    # 既存データの次の行インデックスを計算
    start_row = len(data) + 2  # データ行数 + ヘッダー行

    # スプレッドシートに追記するデータを準備
    processed_data = []
    for key, review_list in reviews.items():
        for review in review_list:
            processed_data.append([title_text, review, one_cel_label_list[key-1], key])  # 必要に応じて他の列も埋める
    print(processed_data)
    # 範囲を指定して一括更新
    range_name = f"A{start_row}:D{start_row + len(processed_data) - 1}"
    worksheet.update(range_name=range_name, values=processed_data)


## 設定

In [ ]:
# ---------------------実行前設定---------------------
# スプレッドシートID
id = '1o0RomqDP_dqUdPLaBGAODMpXXJDRNDdbbUAyksyEJpk'
#id = '1BDa7FUBT4j0FNSz2NgpZt_9Sig0-7ed4D3qBd7_AngY' # 予備
# 拡張対象ラベル番号
aug_label = 6
#手法番号
aug_method_num = 1
important_expression = comparative_expression4
# シート名
sheet_name = f'ラベル{aug_label}-手法{aug_method_num}'
# api1回の生成レビュー数
target_generate_num = 300 #今回の拡張実験に必要なレビュー数256以上でキリのいい300
max_generate_num = 4 #文章が長いと1文の生成が不安定になるため4文に制限
generate_num = math.ceil(target_generate_num / each_label_num[aug_label])

# ---------------------------------------------------

## 手法1,3実行

In [ ]:
# 手法1,3
if generate_num > max_generate_num:
  target_number_of_times = math.ceil(target_generate_num / (each_label_num[aug_label]*max_generate_num))
  generate_num = max_generate_num
  print(f'api1回の生成数 : {max_generate_num}')
  print(f'1実行における生成数 : {(each_label_num[aug_label]*max_generate_num)}')
  print(f'必要なapi実行回数 : {target_number_of_times}')
else:
  target_number_of_times = 1
  print(f'api1回の生成数 : {generate_num}')
  print(f'1実行における生成数 : {(each_label_num[aug_label]*generate_num)}')
  print(f'必要なapi実行回数 : {target_number_of_times}')


message = ""
for i in tqdm(range(target_number_of_times), desc=f'act process'):
  # 拡張するラベルについての既存レビュー抽出
  category_review_dic = show_category_to_review(label_list, review_list, aug_label)
  reviews = {} # 既存のレビューから生成する場合に利用

  for key, review in tqdm(category_review_dic.items(), desc='api process'):
    aug_review, m = generate_aug_review(generate_num, review, aug_method_num, aug_label, important_expression=comparative_expression1)
    divided_aug_review = devide_text(aug_review, generate_num, key)
    reviews[key] = divided_aug_review
    message = m
    time.sleep(1)

  """
  #上記でこの形にする
  reviews = {
      '1': ['レビュー1', 'レビュー2'],
      '5': ['レビュー3', 'レビュー4'],
      '9': ['レビュー5', 'レビュー6']
  }
  """

  write_reviews_to_sheet(id, sheet_name, reviews, one_cel_label_list)

api1回の生成数 : 4
1実行における生成数 : 100
必要なapi実行回数 : 3


api process: 100%|██████████| 25/25 [01:09<00:00,  2.79s/it]


[['5つ星のうち2.5', '使ってみたら楽しかったです！また購入したいと思います。', '5', 38], ['5つ星のうち2.5', 'このバッグはとても嬉しいです。使いやすさも抜群ですね。', '5', 38], ['5つ星のうち2.5', '服を着たらいい感じでした。満足しています。', '5', 38], ['5つ星のうち2.5', 'この靴を履いたら、楽しい気分になりました。良い買い物でした！', '5', 38], ['5つ星のうち2.5', 'このバッグはとても素敵です。', '5', 83], ['5つ星のうち2.5', '使用していて嬉しい気持ちになります。', '5', 83], ['5つ星のうち2.5', 'とても満足しています。', '5', 83], ['5つ星のうち2.5', '使いやすくて楽しいです。', '5', 83], ['5つ星のうち2.5', 'これ、結構良い感じです。', '5', 90], ['5つ星のうち2.5', '使ってみたら、楽しかったです。', '5', 90], ['5つ星のうち2.5', '思ったより嬉しい結果でした。', '5', 90], ['5つ星のうち2.5', 'まあまあ良いと思いますが、詳しくはわからないです。', '5', 90], ['5つ星のうち2.5', 'このバッグは本当に素晴らしいです！', '5', 106], ['5つ星のうち2.5', '使ってみて、とても楽しい気分になりました。', '5', 106], ['5つ星のうち2.5', '商品が届いてからずっと嬉しい気持ちです。', '5', 106], ['5つ星のうち2.5', 'この靴は最高に良いです！', '5', 106], ['5つ星のうち2.5', 'このバッグは最高です！', '5', 125], ['5つ星のうち2.5', 'とても満足しています。', '5', 125], ['5つ星のうち2.5', '本当に嬉しい買い物でした。', '5', 125], ['5つ星のうち2.5', 'このシューズは素晴らしいです！', '5', 125], ['5つ星のうち2.5', '使いやすくて、すごく満足しています。', '5', 166], ['5つ星のうち2.5', 'この商品はとても気に入っています、楽しいです。

api process: 100%|██████████| 25/25 [01:16<00:00,  3.06s/it]


[['5つ星のうち2.5', '使ってみたら、すごく嬉しかったです。とても良い買い物でした。感謝しています。', '5', 38], ['5つ星のうち2.5', 'この服を着てみたら、楽しい気分になりました。素敵な商品ですね。ありがとうございます。', '5', 38], ['5つ星のうち2.5', '使ってみたら、満足感がありました。いい経験でした。また購入したいです。', '5', 38], ['5つ星のうち2.5', 'この靴を履いてみたら、心が弾みました。素晴らしい商品です。感謝です。', '5', 38], ['5つ星のうち2.5', '使ってみたら、すごく楽しい気分になりました。', '5', 83], ['5つ星のうち2.5', 'この服はとても嬉しい気持ちにさせてくれます。', '5', 83], ['5つ星のうち2.5', '商品が届いて、なんだかワクワクしました。', '5', 83], ['5つ星のうち2.5', '使っていると、なんとなく良い感じがします。', '5', 83], ['5つ星のうち2.5', '使ってみたけど、まあまあかな。', '5', 90], ['5つ星のうち2.5', 'デザインは可愛いけど、特に感動はないです。', '5', 90], ['5つ星のうち2.5', '使いやすさは普通だけど、特に良い点は見当たらない。', '5', 90], ['5つ星のうち2.5', '価格は手頃だけど、特に印象に残ることはなかった。', '5', 90], ['5つ星のうち2.5', 'このバッグはとても嬉しいです！', '5', 106], ['5つ星のうち2.5', '使ってみて楽しい気分になります。', '5', 106], ['5つ星のうち2.5', '昨日届いた靴、やっぱり良いです！', '5', 106], ['5つ星のうち2.5', 'このコーヒーメーカー、思った以上に楽しいです。', '5', 106], ['5つ星のうち2.5', 'すごくいい感じ！', '5', 125], ['5つ星のうち2.5', 'とても満足しています。', '5', 125], ['5つ星のうち2.5', '最高の体験でした。', '5', 125], ['5つ星のうち2.5', 'すごく嬉しかったです！', '5', 125], ['5

api process:  64%|██████▍   | 16/25 [00:49<00:27,  3.06s/it]

ID : 541
 4分割失敗



api process: 100%|██████████| 25/25 [01:17<00:00,  3.09s/it]


[['5つ星のうち2.5', '使ってみたらとても楽しかったです！また購入したいと思います。', '5', 38], ['5つ星のうち2.5', 'この服を着たらすごく嬉しかったです。良いお買い物ができました！', '5', 38], ['5つ星のうち2.5', '使った後の感想は、とても満足しています。嬉しい気持ちになりました。', '5', 38], ['5つ星のうち2.5', 'この商品を試したら、楽しい体験ができました。良い選択でした！', '5', 38], ['5つ星のうち2.5', 'このバッグはとても素敵です！', '5', 83], ['5つ星のうち2.5', '本当に楽しい体験でした。', '5', 83], ['5つ星のうち2.5', 'この靴は最高にいい感じです。', '5', 83], ['5つ星のうち2.5', '使っていて嬉しい気持ちになりました。', '5', 83], ['5つ星のうち2.5', 'とても良い感じです。', '5', 90], ['5つ星のうち2.5', '思ったより満足しています。', '5', 90], ['5つ星のうち2.5', '使っていて楽しいです。', '5', 90], ['5つ星のうち2.5', '見た目が素敵で嬉しいです。', '5', 90], ['5つ星のうち2.5', '使ってみたら、やっぱり嬉しい気持ちになった。', '5', 106], ['5つ星のうち2.5', 'この服を着ると、なんだか楽しい気分になる。', '5', 106], ['5つ星のうち2.5', '購入してみて、すごく満足な気持ちになった。', '5', 106], ['5つ星のうち2.5', 'このゲームをプレイしていると、時間を忘れて楽しめる。', '5', 106], ['5つ星のうち2.5', '使ってみたらすごく嬉しかったです。', '5', 125], ['5つ星のうち2.5', 'これ、楽しいですね！', '5', 125], ['5つ星のうち2.5', '最高に良い買い物でした。', '5', 125], ['5つ星のうち2.5', 'とても満足しています！', '5', 125], ['5つ星のうち2.5', 'このバッグは本当に良い感じです。使うのが楽しみです。', '5', 166], ['5つ星のう

act process: 100%|██████████| 3/3 [03:47<00:00, 75.89s/it]


In [ ]:
print(m)

[{'role': 'user', 'content': [{'type': 'text', 'text': '\n                                      ECサイトレビューにおける1つのラベルの定義が下記です。\n                                      \n    良い，悪いなどの簡易的な表現や嬉しい，楽しいなどの単純な表現や感情表現のみで構成されており，具体的な評価対象が含まれていない場合，このラベルが該当する．\n    この簡易表現ラベルでは，評価の根拠となる商品の機能面（使いやすさ，使用感，サイズ，デザインも含む）や価格，配送，商品を利用する状況（使用する頻度や活用方法）の全てについても触れていないことが条件となる．\n    「音質が良い」や「履き心地が良い」，「デザインが良い」などは良い理由となる根拠が含まれているため簡易表現ラベルに該当しない．\n    \n                                      以下は探索材（電化製品，衣類等）における簡易表現ラベルを含むレビュー例です。このように簡易表現を含む文章を4文生成してください。また条件として以下のものを追加します。\n\n                                      １．レビュー例の商品に限らず，別の商品（探索材）のレビューでも構わないです。 ２．ラベルの特徴を反映させたレビューであることは厳守してください。\n                                      良い\n良い\n\n                                      '}]}]


## 手法2実行

In [ ]:
# 手法2
if aug_label == 2:
  generate_num = 5
  target_number_of_times = 15
  print(f'手法2のためapi1回につき{generate_num}文生成')
  print(f'1実行における生成数 : {generate_num*target_number_of_times}')
  review = ""
  for i in tqdm(range(target_number_of_times)):
    reviews = {}
    aug_review, m = generate_aug_review(generate_num, review, aug_method_num, aug_label, important_expression)
    divided_aug_review = devide_text(aug_review, generate_num, i)
    reviews[i] = divided_aug_review
    message = m
    time.sleep(1)
    write_reviews_to_sheet(id, sheet_name, reviews, one_cel_label_list)



手法2のためapi1回につき5文生成
1実行における生成数 : 75


  0%|          | 0/15 [00:00<?, ?it/s]

[['', 'このシャンプーは、断然こちらの方が香りが良くて、使った後の髪のまとまりが全然違います！', '1,4', 0], ['', '比べてみたら、このスニーカーは他のブランドよりもクッション性が優れていて、長時間履いても疲れにくいです。', '1,4', 0], ['', 'ただこの化粧品は、他のものとは違いますが、肌への浸透が早く、使った後のしっとり感が持続します。', '1,4', 0], ['', '他商品と比べて、このスマートフォンのカメラ機能は非常に高性能で、夜景でもきれいに撮影できるのが気に入っています。', '1,4', 0], ['', 'このコーヒーメーカーは、ペーパーフィルタータイプと比べて、手軽に本格的な味わいを楽しめるのが魅力的です。', '1,4', 0]]


  7%|▋         | 1/15 [00:05<01:21,  5.79s/it]

[['', '「この商品は、断然こちらの方が使いやすいです。特に操作が直感的で、初めての私でもすぐに慣れることができました。」', '1', 1], ['', '「他の商品と比べて、このアイテムはコストパフォーマンスが優れています。品質も良く、長持ちするので非常に満足しています。」', '1', 1], ['', '「ただ安いのものとは違いますが、この商品は高品質で耐久性があります。価格は少し高めですが、その価値は十分にあります。」', '1', 1], ['', '「比べてみたら、この商品のデザインが一番気に入りました。シンプルでスタイリッシュな見た目が、どんなインテリアにも合います。」', '1', 1], ['', '「この製品は、従来のタイプと比べて機能が豊富です。特に追加された機能が便利で、日常生活での使い勝手が大幅に向上しました。」', '1', 1]]


...

[['', '「このスマートフォンは、断然こちらの方がカメラの画質が優れています。特に夜景撮影に強く、満足しています。」', '4', 12], ['', '「他商品と比べて、このスニーカーは履き心地が抜群です。長時間歩いても疲れにくく、デザインも気に入っています。」', '4', 12], ['', '「このシャンプーは、ただ普通のものとは違いますが、髪の毛がしっとりとまとまりやすくなりました。リピート確実です。」', '4', 12], ['', '「〜比べてみたら〜、このコーヒーメーカーは抽出時間が短いのに、味は深くてとても満足しています。朝の忙しい時間にぴったりです。」', '4', 12], ['', '「このエコバッグは、他のタイプと比べて耐久性が高く、たくさんの荷物を持っても安心です。デザインもオシャレで気に入っています。」 \n\nこれらの文は、比較を通じて商品の特徴や利点を強調しています。', '4', 12]]


 93%|█████████▎| 14/15 [01:29<00:06,  6.76s/it]

[['', 'この商品は、他商品と比べて使いやすさが抜群です。特に操作性が優れていて、毎日の生活が便利になりました。断然こちらの方が満足度が高いです。', '1,4', 13], ['', '比べてみたら、こちらのアイテムの品質は格段に良いと感じました。特に耐久性があり、長く使えることが実感できます。ただ、値段は少し高めのものとは違いますが、その価値は十分にあります。', '1,4', 13], ['', '他のブランドの製品と比べて、この商品はデザインが洗練されています。見た目がスタイリッシュで、部屋に置いてもおしゃれに見えるのが気に入っています。', '1,4', 13], ['', '〜タイプと比べて、こちらの方が機能性が高いです。この商品を使い始めてから、作業効率が明らかに上がりました。断然こちらの方が選ぶ価値があります。', '1,4', 13], ['', '比べてみたら、以前使っていたものとは違い、この商品は非常に静かです。音が気にならないので、作業に集中できるのが嬉しいです。他商品と比べて、コストパフォーマンスも良いと思います。', '1,4', 13]]


100%|██████████| 15/15 [01:36<00:00,  6.44s/it]

[['', '「このスマートフォンは、他商品と比べてカメラの画質が断然優れています。特に夜景撮影が鮮明で、満足しています。」', '1,2,4', 14], ['', '「価格帯は同じですが、比べてみたらこちらのイヤフォンの音質が圧倒的に良いです。低音の迫力が違います。」', '1,2,4', 14], ['', '「この洗剤は、ただ普通のものとは違いますが、汚れ落ちの効果が素晴らしいです。特に油汚れには強力です。」', '1,2,4', 14], ['', '「このカーテンは、他の商品と比べて遮光性が高く、昼間でも部屋が暗くなります。お昼寝が快適になりました。」', '1,2,4', 14], ['', '「このシャンプーは、他のタイプと比べて髪の潤いが持続します。使った後の指通りが全然違うので、リピート決定です。」', '1,2,4', 14]]


In [ ]:
print(m)

[{'role': 'user', 'content': [{'type': 'text', 'text': 'ECサイトのレビューを利用してファインチューニングを行っています。比較を用いたレビューのデータ拡張を行いたいです。以下の比較を利用してレビューを5文作成してください。\n\n                                      \n〜と比べると〜\n〜より良い，悪い，高い，低い\n〜の方がよかった。\n前のは〜だけど，これは〜\n今まで〜だったけど〜してよかった\n\n                                      '}]}]


## 一部の元レビューから生成


In [ ]:
#1 idを利用した生成するバージョン
#スプレッドシートのapi制限があるので注意
some_review_id = [294, 366, 440]
some_review_dic = {}
for id in some_review_id:
  some_review_dic[id] = review_list[id-1]

print(some_review_dic)

{294: 'コスパは良い\n豆や、粉さえ買えば、自宅でリーズナブルに美味しいコーヒーがいつでも飲めて最高です。\n\n以前はかの有名なデロンギなる物を使用してました。あれは、水やコーヒーの量は勝手に調整してカップ一杯ずつ淹れてくれる優れ物でしたが、定期的な石灰除去なるメンテナンスが必要だし、機械の中の清掃もなかなか複雑でちょっと面倒だったんですよね、、\n\nそれに引き換え、当パナ商品はそこまで汚れを気にする必要はありません。\nとにかく手入れの簡単さが良い点です。\nコーヒー豆さえあれば、安価で自宅で美味しいコーヒーが飲めるのは本当にありがたいですね。\n\n当パナ商品のデメリットは\n毎回水量とコーヒー豆の量を計らなければならない事、カップに直接抽出出来ない事、コーヒーフィルターが必要な事くらいかなー\n\nそれでも、2万前後で、いつでも美味しいコーヒーが飲めるのは非常にポイントは高いですヨ。\n', 366: '余計なことをしない「全自動」コーヒーメーカーの大本命！\n20年来の相棒だったNational製NC-A25からの買い替えで、大(体)満足。\n\n他社製品で1杯用とか2杯用とか、勝手に豆計量してできるようにしてるのがあるけど、その度フィルター掃除とかわたしの中では全自動とは言わない。\nあとお湯が注がれるのが横から見れます、とかあるけど、そんなの見たい人は全自動買わないで自分でネルドリップすると思う。\nその点こいつは放っておいてもわたし好みのコーヒーが「全自動」でできます。大(体)満足だけど、次の点は直した方がいいなぁ。\n\nまず全自動は”豆”から入れたいから選ぶと思うけど、何故か”粉”コースがある。しかも”豆”と”粉”ボタンが同じ大きさで表示の仕方も同じ。朝方の台所で寝ぼけ眼では区別できないのが不満。ボタン大きくするとか色分けするとかはっきりして欲しい。\n次にフィルター上蓋。この蓋をフィルター内側にキチンと入れないとコーヒーがあふれて悲惨になるのだけど、それ防止用なのか”つまみ”が追加された。でも穴に指かけてきっちりフィルターにセットするのでこんなつまみいらない。\n水タンクにフィルターがついているけれどこんなの不要。全自動選ぶような人は浄水器か美味しい水を使っていると思う。\nコーヒーの仕上がりコースが”リッチ”と”マイルド”と”デカフェ

In [ ]:
#2 辞書の半分を生成するバージョン
original_dic = show_category_to_review(label_list, review_list, aug_label)
keys = list(original_dic.keys())

# 半分の長さを計算
half_length = len(keys) // 2

# 前半部分のキーを取得
keys_to_keep = keys#[half_length:]

# 新しい辞書を作成
some_review_dic = {key: original_dic[key] for key in keys_to_keep}

In [ ]:
reviews = {} # 既存のレビューから生成する場合に利用
generate_num = 1 # 元のレビュー1文に追記1文生成
for key, review in tqdm(some_review_dic.items(), desc='api process'):
    aug_review, m = generate_aug_review(generate_num, review, aug_method_num, aug_label, important_expression=comparative_expression1)
    divided_aug_review = devide_text(aug_review, generate_num, key)
    reviews[key] = divided_aug_review
    message = m
    time.sleep(1)

"""
#上記でこの形にする
reviews = {
    '1': ['レビュー1', 'レビュー2'],
    '5': ['レビュー3', 'レビュー4'],
    '9': ['レビュー5', 'レビュー6']
}
"""


api process: 100%|██████████| 26/26 [02:05<00:00,  4.84s/it]


"\n#上記でこの形にする\nreviews = {\n    '1': ['レビュー1', 'レビュー2'],\n    '5': ['レビュー3', 'レビュー4'],\n    '9': ['レビュー5', 'レビュー6']\n}\n"

In [ ]:
write_reviews_to_sheet(id, sheet_name, reviews, one_cel_label_list)

[['5つ星のうち2.5', '迅速な対応に感謝しています。残念ながら届いた商品は片方しか動作しない欠陥品でしたが、メッセージを送ったらすぐに対応していただき、新しい商品が届くのを待っているところです。片方の作動のみで試してみたところ、とても涼しい風が感じられましたが、首にはまだ当てていないため音についてははっきり言えません。ただ、気になるほどではないように思いました。欠陥商品ではありましたが、迅速に対応していただいたので、星は4つにしました。', '1,6,7', 48], ['5つ星のうち2.5', '購入時の相性があると思い、Amazonで購入しましたが、価格に見合った商品だと感じています。ヤマト運輸の配達員の方は、置き配の際に「ここに置いておきますからね」とインターフォンを鳴らしてくれたので、非常に安心して所定の場所に置いて使うことができました。以前は配達時に製品が破損するトラブルがあったため、心配していました。天板の縁は少し薄くて柔らかい印象があり、端に力をかけると壊れそうな気がしますが、天板の上には電子レンジなどの物を置くことができます。音も静かで、24時間中に数回だけモーター音が聞こえます。冷凍庫には霜取りヘラと製氷皿が入っており、年間の電力消費は約¥5000〜6000のようで、経済的にも十分納得しています。全体的に満足しており、針で突いたような小さな傷を見つけたものの、これはもしかしたら几帳面なユーザーが返品した後に再販されたのかもしれないと考えています。以前の一人暮らし用の冷蔵庫は、いろいろあったものの10年以上持ちましたので、頑張ってほしいです。', '1,3,4,6,7', 129], ['5つ星のうち2.5', '冷蔵庫を触るのが少し怖かったです。ダンボールから出して本体に触れた際、静電気に驚かされました。商品の機能や性能は良いと思いますが、冷蔵庫内部の部品には工場での傷がそのまま残っている部分があり、気になる人には気になるかもしれません。また、シールも斜めに貼られていることがありました。大量生産されているので、その気持ちは理解できますが。', '1,6', 145], ['5つ星のうち2.5', 'コストパフォーマンスが良く、付属品も意外に満足しています。私室用に購入したのですが、性能にも満足しています。氷のストッカーや冷凍室の棚は期待して

In [ ]:
print(m)

[{'role': 'user', 'content': [{'type': 'text', 'text': '\n                                      ECサイトレビューにおける1つのラベルの定義が下記です。\n                                      \n    他の商品や同じ商品の前のバージョンと比較を行なっている文が存在する場合，このラベルが該当する．\n    \n                                      以下は探索材（電化製品，衣類）における比較ラベルを含むレビュー例です。このように比較を含む文章を1文生成してください。また条件として以下のものを追加します。\n\n                                      １．レビュー例の商品に限らず，別の商品（探索材）のレビューでも構わないです。 ２．ラベルの特徴を反映させたレビューであることは厳守してください。\n                                      安定感ある タイマー無し問題無し\n十年以上使えたパナソニックのコーヒーメーカーを買い換えるのに、悩みに悩んだ末、またパナソニックの同じタイプの更新版にしました。\n理由は安定感 安心感 衛生的\nトラブルになるのは、紙フィルターがうまく出来てないときに、あふれ出すことがあること。紙フィルター無しだと引き豆かすを捨てるとき粉が散乱して無駄に手間取るし、タイマー式や豆貯蔵式も欲しいときすぐ出来るのはいいが、豆が傷むのでやめました。オフィスとか大家族でバンバン使うならともかく、3人なので。\nタイマー式もセットしたまま放置しておくならと今の我が家スタイルには必要なかったです。\nそれより豆をひいたあとに自動洗浄でミルを流してくれ衛生的で、コンパクトなので、\n安定感安心感があります。\nマイナーチェンジはしてますが、ほぼ同じ機構で何十年と変わらずあることからも機械としての完成度が高いことが分かります。\n\n                                      '}]}]


In [ ]:
reviews